In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType,StringType, IntegerType, StructField, StructType, DoubleType
from delta import *

In [ ]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
spark.sql(
    """
    CREATE DATABASE PL7G1_SILVER LOCATION 'hdfs://hdfs-nn:9000//Poluicao_ar/Silver/PL7G1_SILVER.db/'
    """
)


In [ ]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

In [ ]:
spark.sql(
    """
   SHOW tables from PL7G1_SILVER
    """
).show()


In [ ]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_SILVER.hospital (
        health_ServiceArea VARCHAR(30),
        operating_Certificate_Number VARCHAR(20),
        facilityID VARCHAR(50),
        facility_Name VARCHAR(100),
        age_Group VARCHAR(50),
        zip_Code VARCHAR(50),
        gender VARCHAR(50),
        race VARCHAR(50),
        ethnicity VARCHAR(50),
        length_of_Stay VARCHAR(50),
        type_of_Admission VARCHAR(50),
        patient_Disposition VARCHAR(60),
        ccs_Diagnosis_Code VARCHAR(50),
        ccs_Diagnosis_Description VARCHAR(100),
        ccs_Procedure_Code VARCHAR(50),
        ccs_Procedure_Description VARCHAR(100),
        apr_DRG_Code VARCHAR(50),
        apr_DRG_Description VARCHAR(100),
        apr_MDC_Code VARCHAR(50),
        apr_MDC_Description VARCHAR(100),
        apr_Severity_of_Illness_Code VARCHAR(50),
        apr_Severity_of_Illness_Description VARCHAR(100),
        apr_Risk_of_Mortality VARCHAR(50),
        apr_Medical_Surgical_Description VARCHAR(50),
        emergency_Department_Indicator VARCHAR(50)  
        
    )
    USING DELTA
    PARTITIONED BY (
        discharge_Year INT,
        hospital_County  VARCHAR(50)
    )
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/hospital'
    """
)

In [ ]:
spark.sql(
    """
    SHOW TABLES FROM PL7G1_SILVER
    """
).show()

In [ ]:
spark.sql(
    """
    SELECT *
    FROM hospital
    """
).toPandas()

In [ ]:
spark.sql(
    """
    DESCRIBE FORMATTED hospital
    """
).toPandas()

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2012.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2012 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2012.show(30)
#sales_df.printSchema()


In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2012 = dados2012.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2012 = dados2012.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2012 = dados2012.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2012 = dados2012.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))


In [ ]:
dados2012.filter(dados2012.health_ServiceArea == "Sem Informação").toPandas()

In [ ]:
dados2012 = dados2012.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')


In [ ]:
dados2012.show(30)

In [ ]:
dados2012 = dados2012.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2012 = dados2012.where((col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Queens') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Richmond'))
dados2012 = dados2012.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2012 = dados2012.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

dados2012.show()



In [ ]:
dados2012.select('apr_MDC_Description').count()

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2013.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2013 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2013.show(30)
#sales_df.printSchema()


In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2013 = dados2013.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2013 = dados2013.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2013 = dados2013.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2013 = dados2013.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))


In [ ]:
dados2013 = dados2013.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')


In [ ]:
dados2013.show(30)

In [ ]:
dados2013 = dados2013.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2013 = dados2013.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2013 = dados2013.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))


In [ ]:
dados2013.show(30)

In [ ]:
dados = dados2012.unionByName(dados2013)
dados.show()

In [ ]:
rows = dados2012.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
rows = dados2013.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2014.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2014 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2014.show(30)
#sales_df.printSchema()


In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2014 = dados2014.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2014 = dados2014.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2014 = dados2014.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2014 = dados2014.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2014 = dados2014.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')

In [ ]:
dados2014 = dados2014.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2014 = dados2014.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2014 = dados2014.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
dados2014.show(30)

In [ ]:
rows = dados2014.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2014)
dados.show()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2015.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2015 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2015.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2015 = dados2015.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2015 = dados2015.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2015 = dados2015.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2015 = dados2015.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2015 = dados2015.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')

In [ ]:
dados2015 = dados2015.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2015 = dados2015.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2015 = dados2015.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2015.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2015)
dados.show()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2016.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2016 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2016.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2016 = dados2016.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2016 = dados2016.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2016 = dados2016.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2016 = dados2016.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2016 = dados2016.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')

In [ ]:
dados2016 = dados2016.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2016 = dados2016.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2016 = dados2016.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2016.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2016)
dados.show()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2017.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("abortion_Edit_Indicator", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2017 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2017.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2017 = dados2017.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2017 = dados2017.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2017 = dados2017.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2017 = dados2017.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2017 = dados2017.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight','abortion_Edit_Indicator')

In [ ]:
dados2017 = dados2017.where((col('apr_MDC_Description') == 'Diseases and Disorders of the Respiratory System'))

dados2017 = dados2017.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2017 = dados2017.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2017.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2017)
dados.show()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2018 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2018.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2018 = dados2018.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2018 = dados2018.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2018 = dados2018.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2018 = dados2018.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2018.show(30)

In [ ]:
dados2018 = dados2018.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight')

In [ ]:
dados2018 = dados2018.where((col('apr_MDC_Description') == 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM'))

dados2018 = dados2018.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2018 = dados2018.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2018.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2018)
dados.show()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2019.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2019 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2019.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2019 = dados2019.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2019 = dados2019.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2019 = dados2019.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2019 = dados2019.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2019 = dados2019.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight')

In [ ]:
dados2019 = dados2019.where((col('apr_MDC_Description') == 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM'))

dados2019 = dados2019.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2019 = dados2019.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2019.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2019)
dados.show()

In [ ]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2020.csv"

customSchema = StructType([
    StructField("health_ServiceArea", StringType(), True),
    StructField("hospital_County", StringType(), True),
    StructField("operating_Certificate_Number", StringType(), True),  
    StructField("facilityID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("age_Group", StringType(), True),
    StructField("zip_Code", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("race", StringType(), True),
    StructField("ethnicity", StringType(), True),
    StructField("length_of_Stay", StringType(), True),
    StructField("type_of_Admission", StringType(), True),
    StructField("patient_Disposition", StringType(), True),
    StructField("discharge_Year", IntegerType(), True),
    StructField("ccs_Diagnosis_Code", StringType(), True),
    StructField("ccs_Diagnosis_Description", StringType(), True),
    StructField("ccs_Procedure_Code", StringType(), True),
    StructField("ccs_Procedure_Description", StringType(), True),
    StructField("apr_DRG_Code", StringType(), True),
    StructField("apr_DRG_Description", StringType(), True),
    StructField("apr_MDC_Code", StringType(), True),
    StructField("apr_MDC_Description", StringType(), True),
    StructField("apr_Severity_of_Illness_Code", StringType(), True),
    StructField("apr_Severity_of_Illness_Description", StringType(), True),
    StructField("apr_Risk_of_Mortality", StringType(), True),
    StructField("apr_Medical_Surgical_Description", StringType(), True),
    StructField("payment_Tipology_1", StringType(), True),
    StructField("payment_Tipology_2", StringType(), True),
    StructField("payment_Tipology_3", StringType(), True),
    StructField("birth_Weight", StringType(), True),
    StructField("emergency_Department_Indicator", StringType(), True),
    StructField("total_Costs", StringType(), True),
    StructField("total_Charges", StringType(), True)
       
])

dados2020 = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

dados2020.show(30)
#sales_df.printSchema()

In [ ]:
#substituir os valores em branco por "Sem Informacao"

from pyspark.sql.functions import when, col, concat, lit

dados2020 = dados2020.withColumn(
    "hospital_County",
    when(
        (col("hospital_County").isNull() | (col("hospital_County") == None)), 
        "Sem Informação"
    ).otherwise(col("hospital_County")))


dados2020 = dados2020.withColumn(
    "health_ServiceArea",
    when(
        (col("health_ServiceArea").isNull() | (col("health_ServiceArea") == None)), 
        "Sem Informação"
    ).otherwise(col("health_ServiceArea")))

dados2020 = dados2020.withColumn(
    "apr_Severity_of_Illness_Description",
    when(
        (col("apr_Severity_of_Illness_Description").isNull() | (col("apr_Severity_of_Illness_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Severity_of_Illness_Description")))

dados2020 = dados2020.withColumn(
    "apr_Risk_of_Mortality",
    when(
        (col("apr_Risk_of_Mortality").isNull() | (col("apr_Risk_of_Mortality") == None)), 
        "Sem Informação"
    ).otherwise(col("apr_Risk_of_Mortality")))

In [ ]:
dados2020 = dados2020.drop('total_Charges','total_Costs','payment_Tipology_3','payment_Tipology_2','payment_Tipology_1','zip_Code','birth_Weight')

In [ ]:
dados2020 = dados2020.where((col('apr_MDC_Description') == 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM'))

dados2020 = dados2020.where((col('hospital_County') == 'Queens') | (col('hospital_County') == 'Kings') | (col('hospital_County') == 'Bronx') | (col('hospital_County') == 'Manhattan') | (col('hospital_County') == 'Richmond'))

from pyspark.sql.functions import when, col, concat, lit

dados2020 = dados2020.withColumn(
    "hospital_County",
    when((col("hospital_County") == "Kings"), "Brooklyn").
    when((col("hospital_County") == "Richmond"), "Staten Island").
    otherwise(col("hospital_County")))

In [ ]:
rows = dados2020.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados = dados.unionByName(dados2020)
dados.show()

In [ ]:
dados.printSchema()

In [ ]:
rows = dados.count()
print(f"DataFrame Rows count : {rows}")

In [ ]:
dados \
    .select ("health_ServiceArea", "hospital_County", "operating_Certificate_Number", "facilityID", "facility_Name", "age_Group", "gender", "race", "ethnicity", "length_of_Stay", "type_of_Admission", "patient_Disposition", "discharge_Year", "ccs_Diagnosis_Code", "ccs_Diagnosis_Description", "ccs_Procedure_Code", "ccs_Procedure_Description", "apr_DRG_Code", "apr_DRG_Description","apr_MDC_Code", "apr_MDC_Description", "apr_Severity_of_Illness_Code", "apr_Severity_of_Illness_Description", "apr_Risk_of_Mortality", "apr_Medical_Surgical_Description", "emergency_Department_Indicator") \
    .write \
    .mode("overwrite") \
    .partitionBy("discharge_Year","hospital_County") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/hospital")

#CONFIRMAR SE NESTE SAVE ESTAO TODAS AS COLUNAS

In [ ]:
spark.sql(
    """
    SELECT * FROM PL7G1_SILVER.hospital
    """
).toPandas()